In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
!pip install wget

In [2]:
# Create a dash application
app = dash.Dash(__name__)

In [3]:
# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

In [4]:
!wget - "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2022-12-24 12:25:02--  http://-/
Resolving - (-)... failed: No such host is known. .
wget: unable to resolve host address '-'
--2022-12-24 12:25:05--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: 'spacex_launch_dash.csv.3'

     0K ..                                                    100%  605M=0s

2022-12-24 12:25:06 (605 MB/s) - 'spacex_launch_dash.csv.3' saved [2476/2476]

FINISHED --2022-12-24 12:25:06--
Total wall clock time: 3.9s
Downloaded: 1 files, 2.4K in 0s (605 MB/s)


In [5]:
# Read the spaceX data into pandas dataframe
spacex_launch_dash = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [6]:
spacex_launch_dash.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [7]:
max_payload = spacex_launch_dash['Payload Mass (kg)'].max()
min_payload = spacex_launch_dash['Payload Mass (kg)'].min()

unique_launch_sites = spacex_launch_dash['Launch Site'].unique().tolist()
launch_sites = []
launch_sites.append({'label': 'All Sites', 'value': 'All Sites'})
for launch_site in unique_launch_sites:
    launch_sites.append({'label': launch_site, 'value': launch_site})

In [ ]:
app.layout = html.Div(children=[
    html.Div([
        html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
    ]),
    
    html.Div([
        # TASK 1: Add a Launch Site Drop-down Input Component
        dcc.Dropdown(
                id = 'site-dropdown',
                options = launch_sites,
                placeholder = 'Select a Launch Site here',
                searchable = True ,
                clearable = False,
                value = 'All Sites'
            ),
        # TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
        html.Div(dcc.Graph(id='success-pie-chart')),
    ], style={'padding': '0 30px'}),

    html.Div([
        # TASK 3: Add a Range Slider to Select Payload
        html.Div("Payload range (Kg):", 
            style={'color': '#503D36','font-size': 20, 'padding': '0 30px', 'margin-left': '11px'}
            ),
        html.Div([
            dcc.RangeSlider(
                id = 'payload_slider',
                min = 0,
                max = 10000,
                step = 1000,
                marks = {
                        0: {'label': '0', 'style': {'color': '#77b0b1'}},
                        2500: {'label': '2500'},
                        5000: {'label': '5000'},
                        7500: {'label': '7500'},
                        10000: {'label': '10000', 'style': {'color': '#f50'}},
                },
                value = [min_payload,max_payload]
            ),
        ], style={'padding': '40px 30px'}),

        # TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
        html.Div(dcc.Graph(id = 'success-payload-scatter-chart')),
    ]),
],style={'padding': '0 20px'})

# TASK 2: success-pie-chart callback based on selected site dropdown
@app.callback(
     Output(component_id = 'success-pie-chart', component_property = 'figure'),
     [Input(component_id = 'site-dropdown', component_property = 'value')]
)
def update_piegraph(site_dropdown):
    if (site_dropdown == 'All Sites' or site_dropdown == 'None'):
        all_sites  = spacex_launch_dash [spacex_launch_dash['class'] == 1] # All Success only for all sites.
        fig = px.pie(
                all_sites,
                names = 'Launch Site',
                title = 'Total Success Launches by All Sites',
            )
    else:
        site_specific  = spacex_launch_dash.loc[spacex_launch_dash['Launch Site'] == site_dropdown]
        fig = px.pie(
                site_specific,
                names = 'class',
                title = 'Total Success Launches for Site' +site_dropdown,
            )
    return fig

# TASK 3, 4: Range Slider or Scatter Chart Callback
@app.callback(
     Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
     [Input(component_id = 'site-dropdown', component_property = 'value'), 
     Input(component_id = "payload_slider", component_property = "value")]
)
def update_scattergraph(site_dropdown,payload_slider):
    if (site_dropdown == 'All Sites' or site_dropdown == 'None'):
        low, high = payload_slider
        all_sites  = spacex_launch_dash
        inrange = (all_sites['Payload Mass (kg)'] > low) & (all_sites['Payload Mass (kg)'] < high)
        fig = px.scatter(
                all_sites[inrange], 
                x = "Payload Mass (kg)", 
                y = "class",
                title = 'Correlation Between Payload and Success for All Sites',
                color="Booster Version Category",
                size='Payload Mass (kg)',
                hover_data=['Payload Mass (kg)']
            )
    else:
        low, high = payload_slider
        site_specific  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        inrange = (site_specific['Payload Mass (kg)'] > low) & (site_specific['PAYLOAD_MASS__KG_'] < high)
        fig = px.scatter(
                site_specific[inrange],
                x = "Payload Mass (kg)",
                y = "class",
                title = 'Correlation Between Payload and Success for Site ' +site_dropdown,
                color="Booster Version Category",
                size='Payload Mass (kg)',
                hover_data=['Payload Mass (kg)']
            )
    return fig

if __name__ == '__main__':
    app.run_server()
    app.run_server(mode="inline", host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Dec/2022 12:25:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2022 12:25:26] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2022 12:25:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2022 12:25:26] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2022 12:25:26] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2022 12:25:26] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2022 12:25:26] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2022 12:25:26] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2022 12:25:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2022 12:25:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Dec/2022 12:39:26] "POST /_dash-upd

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\aniek\anaconda3\lib\site-packages\dash\dash.py", line 1274, in dispatch
    ctx.run(
  Fil

127.0.0.1 - - [24/Dec/2022 12:39:26] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [24/Dec/2022 12:39:34] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\aniek\anaconda3\lib\site-packages\dash\dash.py", line 1274, in dispatch
    ctx.run(
  Fil

...

127.0.0.1 - - [25/Dec/2022 20:33:25] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\aniek\anaconda3\lib\site-packages\dash\dash.py", line 1274, in dispatch
    ctx.run(
  Fil

127.0.0.1 - - [25/Dec/2022 20:33:30] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [25/Dec/2022 20:33:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2022 20:33:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2022 20:33:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2022 20:33:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2022 20:33:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2022 20:34:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2022 20:34:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2022 20:34:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2022 20:34:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2022 20:34:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Dec/2022 20:34:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\aniek\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\aniek\anaconda3\lib\site-packages\dash\dash.py", line 1274, in dispatch
    ctx.run(
  Fil

127.0.0.1 - - [26/Dec/2022 00:03:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2022 00:03:11] "POST /_dash-update-component HTTP/1.1" 500 -
